In [1]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from tqdm import tqdm
from flair.data import Sentence
from flair.models import MultiTagger

In [2]:
device = torch.device('cpu')

# Creation of baseline dataset for everyone (1000 samples <512 tokens)

In [5]:
# !pip install datasets

In [6]:
# from datasets import load_dataset

# dataset = load_dataset("xsum")

In [7]:
# data = dataset

# testing_data = data['test'][0:2000]

In [8]:
# testing_data['document'][1]

In [9]:
# testing_data['summary'][1]

In [10]:
# testing_data.keys()

In [11]:
# from flair.data import Sentence
# from flair.models import MultiTagger

# test_set = {'document':[], 'summary': []}


# for count, document in enumerate(testing_data['document'], start=0):
#     text = Sentence(document)
#     if len(text.tokens) < 512:
#         test_set['document'].append(document)
#         test_set['summary'].append(testing_data['summary'][count])
#         if len(test_set['document']) > 999:
#             break

In [12]:
# import json
  
# with open('convert.txt', 'w') as convert_file:
#      convert_file.write(json.dumps(test_set))

# Filter out POS and NER

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("gniemiec/t5-small-finetuned-xsum")

model = AutoModelForSeq2SeqLM.from_pretrained("gniemiec/t5-small-finetuned-xsum")

In [4]:
with open('test_set.txt') as json_file:
    test_set = json.load(json_file)

In [5]:
test_set['document'][2]

'The 48-year-old former Arsenal goalkeeper played for the Royals for four years.\nHe was appointed youth academy director in 2000 and has been director of football since 2003.\nA West Brom statement said: "He played a key role in the Championship club twice winning promotion to the Premier League in 2006 and 2012."'

In [6]:
test_set['summary'][2]

'West Brom have appointed Nicky Hammond as technical director, ending his 20-year association with Reading.'

In [18]:
# !pip install rouge/requirements.txt
# !pip install rouge-score

In [17]:
# from rouge_score import rouge_scorer

# scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
# scores = scorer.score('The quick brown fox jumps over the lazy dog',
#                       'The quick brown dog jumps on the log.')

In [18]:
# scores

{'rouge1': Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765),
 'rougeL': Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471)}

## Run over 1000 samples

In [7]:
from flair.models import MultiTagger
from tqdm import tqdm

# load tagger for POS and NER 
tagger = MultiTagger.load(['ner-fast'])

nouns_filtered = {'document': [], 'summary': test_set['summary']}

for count, document in enumerate(tqdm(test_set['document'])):
    sentence = Sentence(document)
    tagger.predict(sentence)
    
    non_nouns = [token for token in sentence if token.get_tag('upos-fast').value not in ['NOUN','PROPN']]
    
#     for token in sentence:
#         print(token.get_tag('ner-fast'))
    
    
    filtered = []

    for non_noun in non_nouns:
#         print(non_noun)
        filtered.append(non_noun.text)
    
    filtered_sentence = ' '.join(filtered)
    nouns_filtered['document'].append(filtered_sentence)
#     print(filtered_sentence)

2022-04-13 18:18:49,166 --------------------------------------------------------------------------------
2022-04-13 18:18:49,168 The model key 'ner-fast' now maps to 'https://huggingface.co/flair/ner-english-fast' on the HuggingFace ModelHub
2022-04-13 18:18:49,169  - The most current version of the model is automatically downloaded from there.
2022-04-13 18:18:49,169  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner-fast/en-ner-fast-conll03-v0.4.pt)
2022-04-13 18:18:49,170 --------------------------------------------------------------------------------
2022-04-13 18:18:50,343 loading file C:\Users\anujg\.flair\models\ner-english-fast\4c58e7191ff952c030b82db25b3694b58800b0e722ff15427f527e1631ed6142.e13c7c4664ffe2bbfa8f1f5375bd0dced866b8c1dd7ff89a6d705518abf0a611


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [47:15<00:00,  2.84s/it]


In [8]:
len(nouns_filtered['document'])

1000

In [9]:
len(nouns_filtered['summary'])

1000

In [10]:
with open('ORG_filtered.txt', 'w') as convert_file:
     convert_file.write(json.dumps(nouns_filtered))

# Repeat for all POS and NERs

In [ ]:
# object_methods = [method_name for method_name in dir(token)
#                   if callable(getattr(token, method_name))]
# object_methods